# Convertidor Recibos PDF a CSV

Este script extrae las tablas de los recibos de haberes de TASA ubicados en la carpeta `.\Recibos`, los unifica y lo exporta en formato CSV.

Los archivos deben nombrarse con el formato `Recibo AAAA-MM.pdf`.

### Extracción de datos con Tabula-Py

In [119]:
import pandas as pd
import numpy as np
import tabula
import re
from glob import glob

recibos_list = []

for file_name in glob('Recibos\*.pdf'):
    pdf = tabula.read_pdf(file_name, pages='all', multiple_tables=True, stream=True, pandas_options={'header': None},
                         area=(147.0,417.316,344.87,812.004),
                         columns=(449.3, 557.28, 595.4, 638.64, 697.68, 748.08, 811.44))
    
    df_partials = []
    
    for df_part in pdf:
        df_part.columns = ['codigo','concepto','unidad','valor','haberes_s_ret','haberes_c_ret','descuentos']
        df_part = df_part[df_part.codigo != 'Código'] # Elimina header de la página 1
        df_partials.append(df_part)
    
    # Concateno las paginas del recibo y reseteo el índice
    df = pd.concat(df_partials)
    df.reset_index(inplace=True, drop=True)
    
    # Extrae período del nombre del archivo y lo anexo al dataframe
    df['periodo'] = re.search('(\d{4}-\d{2})', file_name).group(1) 
    

    # Convierto tipo de datos para campos numéricos:
    df['unidad'] = pd.to_numeric(df['unidad'].str.replace(',','.'), errors='coerce')
    df['valor'] = pd.to_numeric(df['valor'].str.replace(',','.'), errors='coerce')
    
    df['haberes_s_ret'] = df['haberes_s_ret'].str.replace('(\d*.\d*,\d{2})-','-\\1')
    df['haberes_s_ret'] = df['haberes_s_ret'].str.replace('.','')
    df['haberes_s_ret'] = df['haberes_s_ret'].str.replace(',','.')
    df['haberes_s_ret'] = pd.to_numeric(df['haberes_s_ret'], errors='coerce')
    
    df['haberes_c_ret'] = df['haberes_c_ret'].str.replace('(\d*.\d*,\d{2})-','-\\1')
    df['haberes_c_ret'] = df['haberes_c_ret'].str.replace('.','')
    df['haberes_c_ret'] = df['haberes_c_ret'].str.replace(',','.')
    df['haberes_c_ret'] = pd.to_numeric(df['haberes_c_ret'], errors='coerce')
   
    df['descuentos'] = df['descuentos'].str.replace('(\d*.\d*,\d{2})-','-\\1')
    df['descuentos'] = df['descuentos'].str.replace('.','')
    df['descuentos'] = df['descuentos'].str.replace(',','.')
    df['descuentos'] = pd.to_numeric(df['descuentos'], errors='coerce')

    # Elimino filas importadas con subtotales (recibos viejos un poco más cortos)
    df = df[~df[['codigo','concepto']].isnull().all(1)]
    
    # Sumo recibo completo a la lista
    recibos_list.append(df)

# Concatena todos los recibos
df_recibos = pd.concat(recibos_list)

#display(df_recibos)

### Exporta DataFrame a CSV

In [120]:
df_recibos.to_csv('Recibos.csv', index=False, encoding='ANSI')